<h1>HW4 - Attention and Transformers- Practical Q2</h1>
<h3><font color=yellow>Total Points: 100</font></h3>



<font size=+4>
Introduction
</font>
<br/>
<font size=+2>

<br/>
Part-of-speech tagging (POS tagging) is a natural language processing task where each word in a given text is assigned a specific part-of-speech category, such as noun, verb, adjective, etc. In the context of Roberta, a state-of-the-art language model based on the transformer architecture, POS tagging involves leveraging the model's pre-trained knowledge to accurately predict and label the grammatical roles of words in a sentence. Roberta's extensive training data and attention mechanisms enable it to capture subtle contextual cues, allowing for more nuanced and accurate POS tagging. By understanding the syntactic structure of a sentence, Roberta enhances the efficiency and precision of POS tagging, contributing to more sophisticated language understanding and downstream applications in natural language processing.
</font>


<font size=+2>
In this section we want to fine-tune the Roberta that you trained in previous section for POS-Tagging Task on Parsig dataset
<font/>

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate -U

import re
import torch
import pandas as pd

import numpy as np
import transformers
import matplotlib.pyplot as plt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.5 MB/s eta 0:00:00


In [2]:
# All label that exist
# AAX mean Unknown used for CLS and other symbols
# Added START and END tags for the CLS and CEP tokens. NOTHING is corresponding to the padding tag

tags = {'START','END','NOTHING', 'AAX', 'B-ADJ', 'I-ADJ', 'B-ADV', 'I-ADV', 'B-N', 'I-N', 'B-V', 'I-V',\
        'B-PRONOUN', 'I-PRONOUN', 'B-NUM', 'I-NUM', 'B-DET', 'I-DET', 'B-PRE',\
        'I-PRE', 'B-POST', 'I-POST', 'B-CONJ', 'I-CONJ', 'B-JUNK', 'I-JUNK',\
        'B-MARKER', 'I-MARKER', }
# Convert lables to speceif numeric id
tag2id = {tag: id for id, tag in enumerate(sorted(list(tags)))}
id2tag = {id: tag for tag, id in tag2id.items()}

In [3]:
# Load pretrained model that trained in previous sectiopn
# Load tokenizer from previous section
# Freeze base model for fine-tuning

from transformers import AutoTokenizer, AutoModelForMaskedLM

hugging_face_id = "Dan1212121212"

tokenizer = AutoTokenizer.from_pretrained(f"{hugging_face_id}/mlm_tokenizer")
pretrained_model = AutoModelForMaskedLM.from_pretrained(f"{hugging_face_id}/MLM")

for param in pretrained_model.parameters():
    param.requires_grad = False


tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/65.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/291k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [4]:
# Load datasets
train_data = pd.read_csv("./train_df.csv")
test_data = pd.read_csv("./test_df.csv")
train_data.head()

,text,labels
0,pad nām ī yazdān abdīh ud sahīgīh ī sīstān,"B-PRE,B-N,B-JUNK,B-N,B-N,B-CONJ,B-N,B-JUNK,B-N"
1,abdīh ud sahīgīh ī zamīg ī sīstān az abārīg ša...,"B-N,B-CONJ,B-N,B-JUNK,B-N,B-JUNK,B-N,B-PRE,B-D..."
2,ēk ēn kū rōd ī hēlmand ud war ī frazdān ud zrē...,"B-NUM,B-PRONOUN,B-CONJ,B-N,B-JUNK,B-N,B-CONJ,B..."
3,ud az frazandān ī ēriǰ bē kanīg-ē any kas be n...,"B-CONJ,B-PRE,B-N,B-JUNK,B-N,B-PRE,B-N,I-N,I-N,..."
4,ud pas frēdōn ō war ī frazdān nīd ud pad-nihān...,"B-CONJ,B-ADV,B-N,B-PRE,B-N,B-JUNK,B-N,B-V,B-CO..."


In [5]:
train_data['labels'] = train_data['labels'].str.split(',').apply(lambda x : [tag2id[i] for i in x])
test_data['labels'] = test_data['labels'].str.split(',').apply(lambda x : [tag2id[i] for i in x])

Now we should add some tokens for start and end of the sentence. We will put 27 for the start and 13 for the end in the following cell.

In [6]:
def add_start_end(lst):
    return [27] + lst + [13]

train_data['labels'] = train_data['labels'].apply(add_start_end)
test_data['labels'] = train_data['labels'].apply(add_start_end)

In [7]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_data, test_size=0.2)

In [8]:
print(train_data['labels'].apply(len).max())
print(val_data['labels'].apply(len).max())
print(test_data['labels'].apply(len).max())

381
245
247


As we can see, The maximum length of a sentence is too high and it can make the training hard and inefficient. because the model may struggle to keep track of long term relations like this. So as a limit, I will set the maximum length of sentences to be 64 (I just google searched and this number seemed to be fair and good) and if the sentences are longer than that, I will trunctuate them. This thing will be implemented in the Dataset class.

In [9]:
# Complete custom data set
# Use Torcargmaxh dataloader for datasets

from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df['text'].tolist()
        self.labels = df['labels'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        text = self.texts[index]
        labels = self.labels[index]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')

        # trunctuating the data as we said
        if len(labels) > self.max_len:
            labels = labels[:self.max_len]
        else:
            for _ in range(self.max_len-len(labels)):
                labels.append(26)
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(labels, dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)

In [10]:
max_len = 64

train_dataset = CustomDataset(train_data, tokenizer, max_len)
test_dataset = CustomDataset(test_data, tokenizer, max_len)
val_dataset = CustomDataset(val_data, tokenizer, max_len)

In [11]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

Before training we should do one last job. We have frozen all the layers of the model. now we have to add a fully connected layer to the end of it so that it learns something! In the following cell we will implement this using another class for our final model.

In [12]:
import torch.nn as nn

class FinalModel(nn.Module):
    def __init__(self, frozen_model):
        super(FinalModel, self).__init__()
        self.model = frozen_model.roberta
        self.fc = nn.Linear(frozen_model.config.hidden_size, len(tags))

    def forward(self, input_ids, attention_mask):
        out = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = out.last_hidden_state
        out = self.fc(hidden_states)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FinalModel(pretrained_model).to(device)

In [20]:
import torch.nn.functional as F
from torch import nn
from transformers import AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 10
training_steps = num_epochs * len(train_loader)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, training_steps)

for epoch in range(num_epochs):
    total_loss = 0
    true_preds = 0
    all_preds = 0

    tqdm_obj = tqdm(train_loader)

    for batch in tqdm_obj:
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        loss = criterion(logits.view(-1, model.config.num_labels), labels.view(-1))
        total_loss += loss.item()
        _, predicted = torch.max(logits, 2)
        correct = (predicted == labels).sum().item()
        true_preds += correct
        all_preds += labels.numel()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        tqdm_obj.set_description(f"Epoch (train) {epoch + 1}/{num_epochs} , Acc: {(true_preds / all_preds)*100:.2f}%")

    average_loss = total_loss / len(train_loader)
    print(f"Loss: {average_loss}")

    model.eval()
    total_val_loss = 0
    val_true_preds = 0
    val_all_preds = 0

    with torch.no_grad():
        tqdm_obj = tqdm(val_loader)

        for batch in tqdm_obj:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            loss = F.cross_entropy(logits.view(-1, model.config.num_labels), labels.view(-1), ignore_index=tokenizer.pad_token_id)
            total_val_loss += loss.item()
            _, predicted = torch.max(logits, 2)
            correct = (predicted == labels).sum().item()
            val_true_preds += correct
            val_all_preds += labels.numel()
            tqdm_obj.set_description(f"Epoch (val) {epoch + 1}/{num_epochs} , Acc: {(val_true_preds / val_all_preds)*100:.2f}%")

    average_val_loss = total_val_loss / len(val_loader)
    print(f"Loss: {average_val_loss}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch (train) 1/10 , Acc: 68.67%: 100%|██████████| 41/41 [00:28<00:00,  1.45it/s]


Loss: 1.1860591391237771


Epoch (val) 1/10 , Acc: 82.95%: 100%|██████████| 11/11 [00:02<00:00,  4.43it/s]


Loss: 0.6469117240472273


Epoch (train) 2/10 , Acc: 87.27%: 100%|██████████| 41/41 [00:26<00:00,  1.52it/s]


...

Loss: 0.18579644371162762


Epoch (train) 10/10 , Acc: 97.68%: 100%|██████████| 41/41 [00:26<00:00,  1.55it/s]


Loss: 0.06553451353456916


Epoch (val) 10/10 , Acc: 94.05%: 100%|██████████| 11/11 [00:02<00:00,  4.74it/s]

Loss: 0.19245597178285773


In [24]:
# Report best model accuarcy on validation dataset
# Accuracy below 85% will not be graded
model.eval()
test_true_preds = 0
test_all_preds = 0

with torch.no_grad():
    tqdm_obj = tqdm(test_loader)
    for batch in tqdm_obj:
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        _, predicted = torch.max(logits, 2)
        correct = (predicted == labels).sum().item()
        test_true_preds += correct
        test_all_preds += labels.numel()

test_acc = test_true_preds / test_all_preds
print(f"Test Acc: {test_acc * 100:.2f}%")

100%|██████████| 11/11 [00:02<00:00,  4.40it/s]

Test Acc: 94.04%


In [25]:
# Now implement it with huggingface trainer

from transformers import AdamW, Trainer, TrainingArguments
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

class CustomTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"]
        outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)
        logits = outputs.logits
        loss = criterion(logits.view(-1, 28), labels.view(-1))
        return (loss, outputs) if return_outputs else (loss)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    metric_for_best_model="eval_loss",
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Step,Training Loss,Validation Loss
50,0.214900,4.057918
100,0.180900,4.206613
150,0.161500,4.326836
200,0.152800,4.306776
250,0.136200,4.490425
300,0.128300,4.608558
350,0.116300,4.641980
400,0.107700,4.729490
450,0.099100,4.872318
500,0.091000,4.857745


TrainOutput(global_step=820, training_loss=0.10647693872451783, metrics={'train_runtime': 338.6802, 'train_samples_per_second': 77.3, 'train_steps_per_second': 2.421, 'total_flos': 855294638499840.0, 'train_loss': 0.10647693872451783, 'epoch': 10.0})

And finally we save the model to hugging face!

In [27]:
trainer.save_model("RobertaModel/model_hf.pk")